In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neural_network import MLPClassifier

def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [2]:
location = ''
events = pd.DataFrame.from_csv(location+"user_activity.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"user_activity_test.csv",index_col=None)

In [35]:
events_test.sort_values(by='time', inplace=True)

In [36]:
for i, row in events_test.iterrows():
    if events_test.loc[i,'action'] == 'discovered':
        events_test.set_value(i, 'action', 1)
    if events_test.loc[i, 'action'] == 'passed':
        events_test.set_value(i, 'action', 2)
    if events_test.loc[i, 'action'] == 'viewed':
        events_test.set_value(i, 'action', 3)
    if events_test.loc[i, 'action'] == 'started_attempt':
        events_test.set_value(i, 'action', 4)
    if events_test.loc[i, 'step_type'] == 'choice':
        events_test.set_value(i, 'step_type', 1)
    if events_test.loc[i, 'step_type'] == 'code':
        events_test.set_value(i, 'step_type', 2)
    if events_test.loc[i, 'step_type'] == 'number':
        events_test.set_value(i, 'step_type', 3)
    if events_test.loc[i, 'step_type'] == 'string':
        events_test.set_value(i, 'step_type', 4)
    if events_test.loc[i, 'step_type'] == 'text':
        events_test.set_value(i, 'step_type', 5)
    if events_test.loc[i, 'step_type'] == 'video':
        events_test.set_value(i, 'step_type', 6)
        
    if i % (10 ** 4) == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000


In [91]:
events.to_csv('events_modifyed.csv')

In [9]:
structure.sort_values(['module_position','lesson_position', 'step_position'], inplace=True)

In [10]:
step_id_position = structure.step_id.values.tolist()

In [14]:
step_id_position

[7225,
 7230,
 7231,
 7232,
 43604,
 7233,
 10828,
 2828,
 4848,
 4850,
 4851,
 6219,
 6220,
 6221,
 6222,
 2824,
 2825,
 7257,
 6950,
 2827,
 6951,
 6954,
 6956,
 6957,
 6958,
 6959,
 2829,
 7268,
 6962,
 6963,
 6964,
 6965,
 44097,
 7270,
 6970,
 6969,
 6974,
 6975,
 6976,
 6977,
 2839,
 2832,
 4287,
 39541,
 39540,
 7271,
 6986,
 6985,
 6988,
 6987,
 6984,
 42210,
 4289,
 7299,
 7303,
 7304,
 7305,
 7309,
 2857,
 54850,
 7310,
 7312,
 7311,
 43608,
 41483,
 7322,
 2868,
 40949,
 39542,
 40958,
 42203,
 39598,
 40959,
 40964,
 39605,
 39606,
 40960,
 43605,
 43607,
 43606,
 39597,
 2681,
 6998,
 6999,
 7000,
 7001,
 7002,
 9768,
 7004,
 7003,
 7005,
 7006,
 7007,
 2682,
 4952,
 4955,
 4988,
 9769,
 7009,
 7010,
 7011,
 7012,
 7013,
 2917,
 3124,
 39620,
 9770,
 7019,
 7020,
 7021,
 7022,
 2918,
 2919,
 4969,
 2920,
 2885,
 9771,
 7023,
 7024,
 9810,
 7025,
 7026,
 7027,
 3123,
 3130,
 9772,
 7030,
 9811,
 7029,
 7031,
 7032,
 7033,
 7034,
 3139,
 39630,
 9773,
 7039,
 7040,
 7041,
 7

In [189]:
X = [] #по каждому юзеру имеем век. где параметры : id, набран счет, популяр.рубрика
for us_id in log_progress(set(events.user_id.tolist())):
    temp_us_data = events[events.user_id == us_id]
    temp_us_to_app_vec = []
    
    temp_us_to_app_vec.append(us_id)
    
    temp_us_to_app_vec.append(temp_us_data.step_cost.sum())
    temp_us_to_app_vec.append(temp_us_data.step_cost.mean())
    temp_us_to_app_vec.extend(temp_us_data.step_cost.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.step_type.max())
    temp_us_to_app_vec.append(temp_us_data.step_type.min())
    temp_us_to_app_vec.extend(temp_us_data.step_type.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.time.mean())
    temp_us_to_app_vec.append(temp_us_data.time.max() - temp_us_data.time.min())
    temp_us_to_app_vec.extend(temp_us_data.time.describe().values.tolist())


    temp_us_to_app_vec.extend(temp_us_data.action.describe().tolist())
    
    pos_frame = pd.DataFrame([step_id_position.index(elem) for elem in temp_us_data.step_id])
    for elem in pos_frame.describe().values.tolist():
        temp_us_to_app_vec.append(elem[0])
    
    
    X.append(temp_us_to_app_vec)
Y = []

for elem in log_progress(X):
    Y.append([targets[targets.user_id == elem[0]].passed.values[0]])

In [95]:
X_test = [] #по каждому юзеру имеем век. где параметры : id, набран счет, популяр.рубрика
for us_id in log_progress(set(events_test.user_id.tolist())):
    temp_us_data = events_test[events_test.user_id == us_id]
    temp_us_to_app_vec = []
    
    temp_us_to_app_vec.append(us_id)
    
    temp_us_to_app_vec.append(temp_us_data.step_cost.sum())
    temp_us_to_app_vec.append(temp_us_data.step_cost.mean())
    temp_us_to_app_vec.extend(temp_us_data.step_cost.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.step_type.max())
    temp_us_to_app_vec.append(temp_us_data.step_type.min())
    temp_us_to_app_vec.extend(temp_us_data.step_type.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.time.mean())
    temp_us_to_app_vec.append(temp_us_data.time.max() - temp_us_data.time.min())
    temp_us_to_app_vec.extend(temp_us_data.time.describe().values.tolist())


    temp_us_to_app_vec.extend(temp_us_data.action.describe().tolist())
    
    pos_frame = pd.DataFrame([step_id_position.index(elem) for elem in temp_us_data.step_id])
    for elem in pos_frame.describe().values.tolist():
        temp_us_to_app_vec.append(elem[0])
    
    
    X_test.append(temp_us_to_app_vec)

In [249]:
X_new = pd.DataFrame(X).fillna(method='ffill')
X_test = pd.DataFrame(X_test).fillna(method='ffill')

In [234]:
Y = np.ravel(Y)

In [39]:
def getX(ev):
    'Функция формирует входные вектора для алгоритма RandomForest'
    sc  = ev.groupby('user_id')
    return (sc.sum().index, sc.sum().step_cost.values.reshape((-1, 1)))

In [254]:
#model = DummyClassifier( constant=0)
from sklearn.ensemble import ExtraTreesClassifier
model =  ExtraTreesClassifier(verbose=1, n_jobs = -1, n_estimators=3 * 10 ** 4)

In [319]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(verbose=1, n_jobs = -1, n_estimators = 10)

In [315]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_jobs=-1, n_neighbors=2)

In [328]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
#Поделим нашу выборку на train и validation
Xtr,Xval,Ytr,Yval = train_test_split(X_new,Y,test_size=0.2,random_state=64)

In [329]:
#Обучим модель
model.fit(Xtr, Ytr)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [330]:
#Оценим метрику на validation
print(f1_score(Yval, model.predict(Xval)))

0.366492146597


[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [247]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

In [286]:
#Предскажем результат и запишем его в .csv
ans = model.predict(X_test)
result = np.concatenate((np.asarray(ind_test, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, 'Rand_for_0.48')

[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [304]:
ans.sum()

81.0